# DicoCo

*Sacha Cymermann*

## Importation des librairies

In [2]:
from IPython.display import display, clear_output, HTML, Markdown
from requests_html import HTMLSession
from bs4 import BeautifulSoup
session = HTMLSession()
import re
from pylab import specgram, show
from moviepy.editor import AudioFileClip
from IPython import display as audioD
import IPython.display as ipd
from IPython.display import display, clear_output, HTML, Markdown 
import IPython.display as ipd
import ipywidgets as widgets
import pandas as pd
import numpy as np

pd.options.mode.chained_assignment = None 

## Définition des fonctions : scrapping et audio

In [4]:
def scrapping(df):
    path="https://dictionary.cambridge.org/dictionary/english/"
    listeUK = []
    listeUS = []
    
    for mot in df["motVedette"] :
        link=path+mot
        response = session.get(link)
        status = response.status_code
        html_doc = response.content
        
        soup = BeautifulSoup(response.text,'html.parser')
        souplettesBody = soup.find('body')

        soupletteAside = souplettesBody.findAll('amp-audio')
        soupletteAsideUK = soupletteAside[0].find('source')
        soupletteAsideUS = soupletteAside[1].find('source')

        find = soup.findAll("div", {"class" : "pos-header dpos-h"})

        mp3UK = soupletteAsideUK['src']
        mp3US = soupletteAsideUS['src']

        url='https://dictionary.cambridge.org'
        audioUK = url+mp3UK
        audioUS = url+mp3US
        
        listeUK.append(audioUK)
        listeUS.append(audioUS)
                                  
    df['lienUK']=pd.Series(listeUK).values
    df['lienUS']=pd.Series(listeUS).values
    return df

In [5]:
def Audio(df):
    clear_output()
    listeWid = widgets.Output()
    lU = widgets.Output()
    sU = widgets.Output()

    for numLigne in range(len(df)):
        
        labelUK = widgets.Label("UK")
        itemsUK = audioD.Audio(url = df["lienUK"][numLigne])
        spectroUK = df["lienUK"][numLigne]
    
        labelUS = widgets.Label("US")
        itemsUS = audioD.Audio(url = df["lienUS"][numLigne])
        audioUK = widgets.Output()
        audioUK.append_display_data(itemsUK)
        audioUS = widgets.Output()
        audioUS.append_display_data(itemsUS)
        spectroUS = df["lienUS"][numLigne]
        
        lU = widgets.HBox([labelUK,audioUK,labelUS,audioUS])

        # mots
        labelMot = widgets.Label(df["motVedette"][numLigne])
        labelPronunciation = widgets.Label(str(df["pron"][numLigne]))
        label = widgets.HBox([labelMot, labelPronunciation])


        listeWid.append_display_data(widgets.VBox([label,lU,sU]))
                        
    return listeWid

## Traitement des fichiers dico

In [6]:
cheminDico = "./../data/dictionary/1757_Buchanan-J.csv" 
dfDicoBuch = pd.read_csv(cheminDico, delimiter=";", encoding="utf8")

dfDicoBuch.sample(n=10)

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId
8415,6,mĕdĭtĕrrāʹnėȧn,NaN,NaN,NaN,n.,36.0,138
7571,3,lăʹby̆rinth,F.,NaN,NaN,n.,8.0,126
1653,2,bŭʹrgeſs,NaN,NaN,NaN,n.,36.0,42
2993,2,crāʹdle,NaN,NaN,NaN,n.,14.0,61
4880,2,fāʹbric,NaN,NaN,NaN,n.,4.0,87
9358,3,omĕʹntum,L.,NaN,NaN,n.,50.0,151
8745,4,mŏnŏʹpŏlīze,G.,NaN,NaN,v.,6.0,143
13942,1,tŏn,NaN,NaN,NaN,n.,43.0,215
7745,1,leg,NaN,NaN,NaN,n.,11.0,129
11328,4,rē-ĕxăʹmĭne,NaN,NaN,NaN,v.,16.0,179


In [7]:
pathMap = "./../data/dictionary/Buchanan_PronChar_counts.csv"
dfMap = pd.read_csv(pathMap, delimiter=";", encoding="utf8")

dfMap.sample(n=5)

,char,occ,graph
30,p,3775,p
43,æ,11,æ
5,u,1623,u
63,.,22,NaN
48,у,3,y


In [8]:
dfMap = dfMap.replace(np.nan,'', regex = True)

In [9]:
def correctionTexte(old_word,dfMap):
    new_word = ""
    
    for l in old_word: 
        dfRowFinded = dfMap[dfMap['char'] == l]
        
        if len(dfRowFinded == 1):
            new_word += dfRowFinded.iloc[0]['graph']
            
        else:
            new_word += '?'
            
    return(new_word)

In [10]:
dfDicoBuch['motVedette'] = dfDicoBuch.apply(lambda x: correctionTexte(x['pron'], dfMap),1)

### Enrichissement dictionnaire Buchanan via colonnes Sampa et Ipa

In [12]:
bigi_path="./../data/dataframe/BigiRichSample.csv"
dataBigi = pd.read_csv(bigi_path, delimiter=";", encoding="utf8")

DataDico = dfDicoBuch.merge(dataBigi, how='inner',left_on= 'motVedette', right_on='hwd')
del DataDico['hwd']
DataDico.sample(n=5)

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,motVedette,Sampa,ipa
6453,2,păʹriſh,G.,NaN,NaN,n.,6.0,157,parish,p { r\ i S,pæɹiʃ
160,2,affe˘ʹct,L.,NaN,NaN,v.,50.0,22,affect,@ f E k t,əfɛkt
8598,1,ſquall,NaN,NaN,NaN,n.,69.0,201,squall,s k w O: l,skwOːl
7522,4,rĕfŏrmāʹtion,NaN,NaN,NaN,n.,34.0,179,reformation,r\ E f 3:r m eI S @ n,ɹɛfɜ:rm?ʃən
3342,4,făcēʹti_o_us,F.,NaN,NaN,a.,7.0,87,facetious,f @ s i: S @ s,fəsiːʃəs


In [13]:
DataDico.to_csv("./../data/dataframe/BuchananRich.csv")

In [14]:
Buch_path = "./../data/dataframe/BuchananRich.csv"
Buch = pd.read_csv(Buch_path, delimiter=",", encoding="utf8")
Buch.head(5)

,Unnamed: 0,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,motVedette,Sampa,ipa
0,0,3,aʹbăcus,NaN,NaN,NaN,n.,2.0,19,abacus,{ b @ k @ s,æbəkəs
1,1,3,ābăʹndon,F.,NaN,NaN,v.,5.0,19,abandon,@ b { n d @ n,əbændən
2,2,4,ābăʹndŏned,NaN,NaN,NaN,a.,6.0,19,abandoned,@ b { n d @ n d,əbændənd
3,3,2,ăbăʹſh,F.,NaN,NaN,v.,9.0,19,abash,@ b { S,əbæʃ
4,4,2,ăbāʹte,F.,NaN,NaN,v.,10.0,19,abate,@ b eI t,əb?t


In [15]:
dataBuch=Buch[["motVedette","nbSyll","pron","Sampa","ipa"]]
dataBuch.head(3)

,motVedette,nbSyll,pron,Sampa,ipa
0,abacus,3,aʹbăcus,{ b @ k @ s,æbəkəs
1,abandon,3,ābăʹndon,@ b { n d @ n,əbændən
2,abandoned,4,ābăʹndŏned,@ b { n d @ n d,əbændənd


In [16]:
filtreHWD = widgets.Text(value="",description='filtre hwd',layout={'width': '200px'} )
filtreIPA = widgets.Text(value="",description='filtre ipa',layout={'width': '200px'} )
occMax = widgets.IntSlider(min=0,max=15,step=1,description='Occ.Max:',value=1)
button = widgets.Button(description='GO', icon = "check")

In [17]:
def on_button_clicked(_):
    nbLigne = occMax.value
    HWD = filtreHWD.value 
    IPA = filtreIPA.value
    df = ""

    with dataFrame:
        clear_output()
        if (HWD != ""):
            df = dataBuch[(dataBuch.motVedette.str.match(HWD))]
            if(IPA != ""):
                df = df[(df.pron.str.match(IPA))]
        elif (IPA != ""):
            df = dataBuch[(dataBuch.pron.str.match(IPA))]

        if(len(df) > nbLigne):
            df = df.sample(n=nbLigne).copy()
        df = df.reset_index()    
        display(df)
        
    with audio :
        clear_output()
        df = scrapping(df)  
        
        listeAudio = Audio(df) 
        display(listeAudio)
        
            
button.on_click(on_button_clicked)

In [18]:
dataFrame = widgets.Output()
audio = widgets.Output()

## Affichage de l'interface utilisateur

In [19]:
box1 = widgets.HBox([filtreHWD, filtreIPA, occMax, button])
box2 = widgets.HBox([dataFrame])
box3 = widgets.HBox([audio])

UI = widgets.VBox([box1, box2, box3])
UI

## Spectrogramme

J'ai testé d'afficher les spectrogrammes associés aux sons du scrapping mais je n'ai pas abouti sur quelque chose de fonctionnel.

In [20]:
import matplotlib.pyplot as plt

import librosa
import librosa.display

filename="https://dictionary.cambridge.org/media/english/us_pron/l/lov/love_/love.mp3"
y, sr = librosa.load(filename)

D = librosa.stft(y)  # STFT of y
S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)

fig, ax = plt.subplots()
img = librosa.display.specshow(S_db, x_axis='time', y_axis='log', ax=ax)
ax.set(title='Spectre avec des axes logarithmiques')
fig.colorbar(img, ax=ax, format="%+2.f dB")

OSError: [Errno 22] Invalid argument: 'https://dictionary.cambridge.org/media/english/us_pron/l/lov/love_/love.mp3'